<h1 align='center'> Challenge Presentation </h1>

<div align='center'>
<img src="..\..\..\01_aux_files\img\UK_Traffic_Data\7.png" height='20%' width='30%'/>
<img src="..\..\..\01_aux_files\img\UK_Traffic_Data\6.png" height='20%' width='30%'/>
</div>

## We will create the following tables for our Power Bi Report
* dim_region
* dim_vehicle
* fact_road_vehicle_count
### We will save them to the results folder

In [31]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col, to_date, split, weekofyear, month, lpad, when, lit, regexp_replace, count, avg, year, row_number, sum

In [32]:
spark = SparkSession.builder.appName("myapp").getOrCreate()

## Creating and storing our dim_date

In [33]:
import pandas as pd;

In [34]:
dates = pd.date_range(start='2000-01-01', end='2024-12-31')

df = pd.DataFrame(
    {
    'date_id': dates.strftime('%Y%m%d').astype(int),
    'month': dates.strftime('%m'),
    'year': dates.year,
    'date': dates
    }
)  
df

,date_id,month,year,date
0,20000101,01,2000,2000-01-01
1,20000102,01,2000,2000-01-02
2,20000103,01,2000,2000-01-03
3,20000104,01,2000,2000-01-04
4,20000105,01,2000,2000-01-05
...,...,...,...,...
9127,20241227,12,2024,2024-12-27
9128,20241228,12,2024,2024-12-28
9129,20241229,12,2024,2024-12-29
9130,20241230,12,2024,2024-12-30


In [35]:
df.to_parquet(r'..\..\..\01_aux_files\results\UK_Traffic_Data\Power_BI\dim_time\dim_time.parquet')
print("dim_time stored!")

dim_time stored!


## Extracting the main source

In [36]:
df = spark.read.parquet(r'..\..\..\01_aux_files\source\UK_Traffic_Data\dft_traffic_counts_raw_counts\dft_traffic_counts_raw_counts.parquet')
new_columns = [col(column).alias(column.lower()) for column in df.columns]
df = df.select(*new_columns)

In [37]:
df.show()

+--------------+-------------------+----+-------------------+----+---------+-----------+---------------+------------------+--------------------+--------------------+---------+-------------+---------+------------------------+----------------------+-------+--------+------------+------------+--------------+-------------------+------------+--------------------------+--------------+-----------------+----+-----------------+-----------------+-------------------------+----------------------------+-----------------------+-----------------------+--------+------------------+
|count_point_id|direction_of_travel|year|         count_date|hour|region_id|region_name|region_ons_code|local_authority_id|local_authority_name|local_authority_code|road_name|road_category|road_type|start_junction_road_name|end_junction_road_name|easting|northing|    latitude|   longitude|link_length_km|  link_length_miles|pedal_cycles|two_wheeled_motor_vehicles|cars_and_taxis|buses_and_coaches|lgvs|hgvs_2_rigid_axle|hgvs_3_r